# Portfolio Construction 1

## Objectives
1. Create a DataFrame which contains returns of multiple assets
2. Create functions to convert prices into returns and to add new columns to the dataframe

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Portfolio_construction_1_returns_dataset as pc

## Returns Dataset

Create a function that will clean and give the close column from the OHLC data

In [3]:
def get_close_price_df(OHLC_file_path, Column_name = 'Close'):
    """
    converts an OHLC csv file into a dataframe and then returns the close column along with date column as index
    Best for higher time frame datasets like daily and weekly because it has no time section in Datetime column
    """
    df = pd.read_csv(OHLC_file_path)
    df = df[["Date", "Close"]]
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.set_index("Date")
    df = df.rename(columns={'Close':Column_name})

    return df


In [4]:
def change_timeframe(df, Timeframe, aggregation='sum'):
    """
    Takes a datetime dataframe and resamples it's every column into the given timeframe

    inputs:
    df - Dataframe
    Timeframe - New timeframe fot the dataset
    aggregation - by default 'last', but can change according to the need
    """
    column_names = df.columns
    aggregation_dict = {column: aggregation for column in column_names}
    resampled_df = df.resample(Timeframe).agg(aggregation_dict)
    return resampled_df

```python
def clean_dividends_df(file_path, new_column_name):
    """
    Takes a csv file containing dividends release date and dividends amount
    """
    df = pd.read_csv(file_path)
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)
    df = df.rename(columns={df.columns[0]:new_column_name})
    return df

```

```python
def get_four_major_asset_classes():
    """
    Gives the dividends adjusted returns of four major asset classes -> Gold, Real Estate(VNQ.mx), Bonds(BND.mx) and Equity(VTI.mx)
    """

    GLD = get_close_price_df('Data\OHLC_data\SPDR_Gold_Shares _(GLD).csv', 'Gold')
    VNQ = get_close_price_df('Data\OHLC_data\Vanguard_Real_Estate_Index-Fund_(VNQ).csv', 'Real Estate')
    BND = get_close_price_df('Data\OHLC_data\Vanguard_Total_Bond_Market_Index_Fund_(BND).csv', 'Bonds')
    VTI = get_close_price_df('Data\OHLC_data\Vanguard_Total_Market_Index_Fund_(VTI).csv', 'Equity')

    BND_div = clean_dividends_df('Data\Dividends_data\BND.csv', 'Bonds Div')
    VNQ_div = clean_dividends_df('Data\Dividends_data\VNQ.csv', 'Real Estate Div')
    VTI_div = clean_dividends_df('Data\Dividends_data\VTI.csv', 'Equity Div')

    dfs = [GLD, VNQ, BND, VTI]
    df = pd.concat(dfs, axis=1)
    df = pd.merge_asof(df.sort_index(), change_timeframe(BND_div, 'W').sort_index(),
                          left_index=True, right_on='Date', direction='nearest', tolerance=pd.Timedelta(days=7))
    df = pd.merge_asof(df.sort_index(), change_timeframe(VNQ_div, 'W').sort_index(),
                          left_index=True, right_on='Date', direction='nearest', tolerance=pd.Timedelta(days=7))
    df = pd.merge_asof(df.sort_index(), change_timeframe(VTI_div, 'W').sort_index(),
                          left_index=True, right_on='Date', direction='nearest', tolerance=pd.Timedelta(days=7))
    df.fillna(0, inplace=True)
    df['Real Estate'] = df['Real Estate'] + df['Real Estate Div'].cumsum()
    df['Bonds'] = df['Bonds'] + df['Bonds Div'].cumsum()
    df['Equity'] = df['Equity'] + df['Equity Div'].cumsum()
    df = df.drop(columns=['Real Estate Div', 'Bonds Div', 'Equity Div'])
    df = df.pct_change().dropna()

    return df


def annualize_returns(r, periods_per_year):
    """
    Annualizes a set of return
    """

    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year/n_periods) - 1

```

In [5]:
returns = pc.get_four_major_asset_classes()
returns.head()

,Gold,Real Estate,Bonds,Equity
Date,,,,
2011-02-07,0.005013,0.025767,0.002451,0.016861
2011-02-14,0.023352,0.003589,0.004389,0.011491
2011-02-21,0.014548,-0.005449,0.007366,-0.017256
2011-02-28,0.014340,-0.010101,-0.004338,0.001756
2011-03-07,-0.008109,0.000346,0.008116,-0.014169


In [6]:
returns.corr()

,Gold,Real Estate,Bonds,Equity
Gold,1.000000,0.232321,0.390398,0.122310
Real Estate,0.232321,1.000000,0.318487,0.757416
Bonds,0.390398,0.318487,1.000000,0.040619
Equity,0.122310,0.757416,0.040619,1.000000


In [7]:
returns.std()

Gold           0.021004
Real Estate    0.023728
Bonds          0.005197
Equity         0.021594
dtype: float64

In [8]:
returns.mean()

Gold           0.000689
Real Estate    0.001411
Bonds          0.000373
Equity         0.002221
dtype: float64

In [9]:
annualized_returns = pc.annualize_returns(change_timeframe(returns,'M'), 12)
annualized_returns

Gold           0.023895
Real Estate    0.064604
Bonds          0.018957
Equity         0.111132
dtype: float64

In [10]:
annualized_vol = pc.annualize_vol(returns, 52)
annualized_vol

Gold           0.151462
Real Estate    0.171105
Bonds          0.037474
Equity         0.155718
dtype: float64

In [11]:
annualized_returns/annualized_vol

Gold           0.157763
Real Estate    0.377570
Bonds          0.505878
Equity         0.713675
dtype: float64